In [5]:
import torch
import torchvision
from torch.utils.mobile_optimizer import optimize_for_mobile

model = torchvision.models.mobilenet_v2(pretrained=True)
model.eval()
example = torch.rand(1, 3, 224, 224)
traced_script_module = torch.jit.trace(model, example)
traced_script_module_optimized = optimize_for_mobile(traced_script_module)      
traced_script_module_optimized._save_for_lite_interpreter("app/src/main/assets/model.ptl")  # 해당 코드 실행하면 app/src/main/assets dir에 model.ptl 생성됨

- 생성된 가중치파일이 Android app에서 asset으로 작동함
![](images/2022-02-18-14-09-23.png)

- build.gradle 의존성 셋팅


```xml
repositories {
    jcenter()
}

dependencies {
    implementation 'org.pytorch:pytorch_android_lite:1.9.0'     // Pytorch Android API
    implementation 'org.pytorch:pytorch_android_torchvision:1.9.0'      // android.media.Image -> android.graphics.Bitmap으로 바꿔주는 기능
}
```

- png, jpg 이미지 안드로이드 Bitmap 객체로 변환
  
```java
Bitmap bitmap = BitmapFactory.decodeStream(getAssets().open("image.jpg"));
```

- **가중치 파일** 로드

```java
Module module = Module.load(assetFilePath(this, "model.ptl"));
```

- Bitmap 객체 32bit float Tensor로 변환 및 전처리
  - 주의사항으로 로드된 가중치파일이 학습할 때 아래 코드와 같은 `전처리`가 이루어진것으로 해야 성능 하락을 억제할 수 있음
```java
Tensor inputTensor = TensorImageUtils.bitmapToFloat32Tensor(bitmap,
    TensorImageUtils.TORCHVISION_NORM_MEAN_RGB, TensorImageUtils.TORCHVISION_NORM_STD_RGB);
    // mean: ([0.485, 0.456, 0.406]) std: ([0.229, 0.224, 0.225])임. 얘네가 Imagenet 전체 데이터에 계산한 평균, 표준편차 값임
    // 이미지 사이즈는 자동으로 정해주나..?
```

- Inference

```java
Tensor outputTensor = module.forward(IValue.from(inputTensor)).toTensor();
float[] scores = outputTensor.getDataAsFloatArray();        // pretrained된 imagenet으로 진행했다면 (1, 1000)으로 output 나옴

float maxScore = -Float.MAX_VALUE;      
int maxScoreIdx = -1;
for (int i = 0; i < scores.length; i++) {       // Inference 값 중 가장 큰 값의 index 구함
  if (scores[i] > maxScore) {
    maxScore = scores[i];
    maxScoreIdx = i;
  }
}
String className = ImageNetClasses.IMAGENET_CLASSES[maxScoreIdx];
```

- 의문사항
  - 왜 img, ptl을 assets 폴더에 두는거지?
  - 평균이 0.5가 아니라 저렇게 나오는 이유가 있나..? 표준 정규화 거치면 무조건 평균 0, 표준편차 1로 되는거 아님?